<a href="https://colab.research.google.com/github/bhaweshiitk/6.869_project/blob/main/dropout_model_6_869_project_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
use_gdrive = False

In [2]:
import os
from tqdm import tqdm

In [3]:
! mkdir ~/.kaggle

In [4]:
! cd ~/.kaggle

In [5]:
# First download Kaggle.json from kaggle containing your credentials.
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d kmader/rsna-bone-age

100% 9.28G/9.29G [02:52<00:00, 58.6MB/s]
100% 9.29G/9.29G [02:52<00:00, 57.7MB/s]


In [7]:
!mkdir /content/data
!unzip /content/rsna-bone-age.zip -d /content/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4264.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4265.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4266.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4268.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4269.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4270.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4271.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4272.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4273.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-dataset/4275.png  
  inflating: /content/data/boneage-training-dataset/boneage-training-

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
# You might not have tqdm, which gives you nice progress bars
!pip install tqdm
from tqdm.notebook import tqdm
import os
import copy
import pandas as pd
import PIL 
from google.colab import files  
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")
    print("You may want to try to use the GPU in Google Colab by clicking in:")
    print("Runtime > Change Runtime type > Hardware accelerator > GPU.")

Using the GPU!


In [9]:
class ImageModel(torch.nn.Module):

  def __init__(self, pretrained=True):
    super(ImageModel, self).__init__()
    model_ft = models.resnet18(pretrained=pretrained)
    num_ftrs = model_ft.fc.in_features
    # model_ft.fc = nn.Sequential(nn.Linear(512, 1))
    model_ft.fc = nn.Sequential(nn.Linear(512, 256), nn.Dropout(p=0.2), # Change dropout here
                             nn.ReLU(), nn.Linear(256, 64), nn.Dropout(p=0.2),
                             nn.ReLU(), nn.Linear(64, 1))
    self.model = model_ft # Vggnet, MSE, Pinball loss
    
    # self.input_size = (224, 224)

  def forward(self, x):
    return self.model(x)

In [10]:
from torchvision.transforms.functional import to_grayscale

def get_image_transforms():
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # we convert the image to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from aggregating lots of data and happen to produce better results.
    transform = transforms.Compose([
            transforms.Resize(image_size, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomRotation(degrees=(-10, 10), expand=False),
            transforms.RandomAffine(degrees=(-10, 10), scale=(0.8,1.2)), 
	          transforms.RandomHorizontalFlip(p=0.15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.1871, 0.1871, 0.1871],std=[0.1401, 0.1401, 0.1401])
            
        ])
    return transform

# import glob
from PIL import Image

class ImageDataset(torch.utils.data.Dataset):

  def __init__(self, df, image_dir):
    self.image_paths = []
    self.bone_ages = []
    self.image_ids = []
    for image_id, boneage in zip(df['id'].values, df['boneage'].values):
      self.image_paths.append(f"{image_dir}/{image_id}.png")
      self.bone_ages.append(boneage)
      self.image_ids.append(image_id)
    self.transform = get_image_transforms()
  
  def __len__(self):
    return len(self.image_paths)
  
  def __getitem__(self, idx):
    filepath = self.image_paths[idx]
    boneage = self.bone_ages[idx]
    image_id = self.image_ids[idx]
    image = Image.open(filepath)
    image = self.transform(image)

    return {'image': image, 'age': boneage, 'id':image_id}

In [11]:
def make_optimizer(model, learning_rate, print_parameters=False):
    # Get all the parameters
    params_to_update = list(model.parameters())
    if print_parameters:
      print("Params to learn:")
      for name, param in model.named_parameters():
          if param.requires_grad == True:
              print("\t",name)

 
    optimizer = optim.SGD(params_to_update, lr=learning_rate, momentum = 0.9)
    return optimizer

def get_loss():
    # Create an instance of the loss function
    # criterion = nn.CrossEntropyLoss()
    criterion = nn.functional.mse_loss
    return criterion

In [12]:

def train_model(model, dataloaders, criterion, optimizer, save_dir = None, save_all_epochs=False, num_epochs=25):
    '''
    model: The NN to train
    dataloaders: A dictionary containing at least the keys 
                 'train','val' that maps to Pytorch data loaders for the dataset
    criterion: The Loss function
    optimizer: The algorithm to update weights 
               (Variations on gradient descent)
    num_epochs: How many epochs to train for
    save_dir: Where to save the best model weights that are found, 
              as they are found. Will save to save_dir/weights_best.pt
              Using None will not write anything to disk
    save_all_epochs: Whether to save weights for ALL epochs, not just the best
                     validation error epoch. Will save to save_dir/weights_e{#}.pt
    '''
    since = time.time()

    val_acc_history = []
    train_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            # running_corrects = 0

            # Iterate over data.
            # TQDM has nice progress bars
            for batch in tqdm(dataloaders[phase]):
                image = batch['image'].to(device)
                age = batch['age'].to(device)
                # inputs = inputs.to(device)
                # labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(image)
                    loss = criterion(outputs.squeeze(), age.to(torch.float32), reduction='mean')
                    print(loss)
                    # torch.max outputs the maximum value, and its index
                    # Since the input is batched, we take the max along axis 1
                    # (the meaningful outputs)
                    # _, preds = torch.max(outputs, 1)

                    # backprop + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * image.size(0)
                # running_corrects += torch.sum(preds == age.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            # epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            

            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train':
                train_acc_history.append(epoch_loss)
            if phase == 'val':
                val_acc_history.append(epoch_loss)
            if save_all_epochs:
                torch.save(model.state_dict(), os.path.join(save_dir, f'weights_{epoch}.pt'))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # save and load best model weights
    torch.save(best_model_wts, os.path.join(save_dir, 'weights_best_val_acc.pt'))
    torch.save(model.state_dict(), os.path.join(save_dir, 'weights_last.pt'.format(epoch)))
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history

In [13]:
from pathlib import Path
data_dir = "./data"
models_dir = data_dir + '/models'
Path(models_dir).mkdir(parents=True, exist_ok=True)
save_dir = models_dir + '/trained_model_1'
os.makedirs(save_dir, exist_ok=True)

In [14]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
# You can add your own, or modify these however you wish!
# model_name = "vgg"###Your own###

# Number of classes in the dataset, normal, benign, malignant
# num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Shuffle the input data?
shuffle_datasets = True

# Number of epochs to train for 
num_epochs = 60 ###Your own###

# Learning rate
learning_rate = 0.00005 ###Your own###

### IO
# Path to a model file to use to start weights at
resume_from = None

# Save all epochs so that you can select the model from a particular epoch
save_all_epochs = False

# Whether to use early stopping (load the model with best accuracy), or not
early_stopping = True

# Dataset directory
image_dir = "/content/data/boneage-training-dataset/boneage-training-dataset"
dataframe_path = "/content/data/boneage-training-dataset.csv"

# Image size
image_size = (224, 224)

# # Directory to save weights to
save_dir = models_dir + '/trained_model_1'
os.makedirs(save_dir, exist_ok=True)


In [15]:
import pandas as pd


df = pd.read_csv(dataframe_path)
df = df.sample(frac=1)
train_df = df.iloc[:6000]
val_df = df.iloc[6000:7000]
test_df = df.iloc[7000:]


In [ ]:
# ids_train = list(train_df.id)
# ids_test = list(test_df.id)
# ids_val = list(val_df.id)
# len(ids_train), len(ids_test), len(ids_val)

In [ ]:
# import pickle
# # with open('ids_train.pkl', 'wb') as f:
# #   pickle.dump([ids_train, ids_test, ids_val], f)
# with open('ids_train.pkl', 'rb') as f:
#   id_list = pickle.load(f)
# train_df = df[df.id.isin(id_list[0])]
# test_df = df[df.id.isin(id_list[1])]
# val_df = df[df.id.isin(id_list[2])]

In [16]:
train_dataset = ImageDataset(train_df, image_dir)
val_dataset = ImageDataset(val_df, image_dir)
test_dataset = ImageDataset(test_df, image_dir)
len(train_dataset), len(test_dataset), len(val_dataset)

(6000, 5611, 1000)

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
dataloader = {"train": train_dataloader, "val": val_dataloader, "test": test_dataloader}

In [18]:
# count = 0
# for batch in dataloaders['boneage-test-dataset']:
#   # print(batch)
#   count += batch[0].shape[0]

In [19]:
model = ImageModel(pretrained=False)
model.to(device)

ImageModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [ ]:
# model_1, input_size = initialize_model(model_name = model_name, num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
# dataloaders = get_dataloaders(mias_dataset_dir, input_size, batch_size, shuffle_datasets)
criterion = get_loss()

# Move the model to the gpu if needed
# model_1 = model.to(device)

optimizer_1 = make_optimizer(model, learning_rate=learning_rate)

# Train the model!
trained_model_1, validation_history_1, train_history_1 = train_model(model=model, 
                                                                     dataloaders=dataloader, 
                                                                     criterion=criterion, 
                                                                     optimizer=optimizer_1,
                                                                     save_dir=save_dir, 
                                                                     save_all_epochs=save_all_epochs, 
                                                                     num_epochs=num_epochs)
del model, optimizer_1, trained_model_1

Epoch 1/20
----------


  0%|          | 0/188 [00:00<?, ?it/s]

tensor(485.7150, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(590.4205, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(601.6787, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(803.6199, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(357.8937, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(659.9370, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(672.0325, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(779.6741, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(671.9200, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(637.1882, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(772.5233, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(403.1878, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(943.0424, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(373.6034, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(523.7856, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(753.2164, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(7

  0%|          | 0/32 [00:00<?, ?it/s]

tensor(2209.2505, device='cuda:0')
tensor(1956.5753, device='cuda:0')
tensor(668.3201, device='cuda:0')
tensor(635.8358, device='cuda:0')
tensor(841.7213, device='cuda:0')
tensor(602.6553, device='cuda:0')
tensor(844.3104, device='cuda:0')
tensor(980.6354, device='cuda:0')
tensor(752.9679, device='cuda:0')
tensor(465.0081, device='cuda:0')
tensor(1078.5156, device='cuda:0')
tensor(862.0917, device='cuda:0')
tensor(929.4397, device='cuda:0')
tensor(1001.3746, device='cuda:0')
tensor(1312.5728, device='cuda:0')
tensor(796.2592, device='cuda:0')
tensor(869.1938, device='cuda:0')
tensor(972.5599, device='cuda:0')
tensor(594.3924, device='cuda:0')
tensor(857.3872, device='cuda:0')
tensor(536.8303, device='cuda:0')
tensor(473.7140, device='cuda:0')
tensor(1038.1497, device='cuda:0')
tensor(592.1258, device='cuda:0')
tensor(1399.1545, device='cuda:0')
tensor(1139.0858, device='cuda:0')
tensor(810.5721, device='cuda:0')
tensor(750.5706, device='cuda:0')
tensor(726.0801, device='cuda:0')
tensor

...

tensor(1239.3401, device='cuda:0')
tensor(510.5858, device='cuda:0')
tensor(735.7507, device='cuda:0')
tensor(782.0267, device='cuda:0')
tensor(575.2018, device='cuda:0')
tensor(874.3915, device='cuda:0')
tensor(321.3372, device='cuda:0')
tensor(368.3624, device='cuda:0')
tensor(185.3423, device='cuda:0')
tensor(326.1389, device='cuda:0')
tensor(475.4730, device='cuda:0')
tensor(554.9232, device='cuda:0')
tensor(258.7485, device='cuda:0')
tensor(283.5959, device='cuda:0')
tensor(274.5439, device='cuda:0')
tensor(333.9351, device='cuda:0')
tensor(391.1228, device='cuda:0')
tensor(363.0359, device='cuda:0')
tensor(313.7503, device='cuda:0')
tensor(235.8362, device='cuda:0')
tensor(423.9695, device='cuda:0')
tensor(305.1800, device='cuda:0')
tensor(405.7095, device='cuda:0')
tensor(269.4347, device='cuda:0')
tensor(480.7361, device='cuda:0')
tensor(440.0251, device='cuda:0')
tensor(198.3487, device='cuda:0')
tensor(434.7961, device='cuda:0')
tensor(581.2252, device='cuda:0')
tensor(313.20

  0%|          | 0/188 [00:00<?, ?it/s]

tensor(715.2408, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(305.5605, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(433.0699, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(560.7145, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(407.6928, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(624.1744, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(376.1104, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(547.3158, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(514.3271, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(296.7914, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(329.6934, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(289.7300, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(361.8477, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(605.5015, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(353.5480, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(499.2801, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(4

  0%|          | 0/32 [00:00<?, ?it/s]

tensor(758.6187, device='cuda:0')
tensor(656.5195, device='cuda:0')
tensor(427.0017, device='cuda:0')
tensor(457.4674, device='cuda:0')
tensor(325.6287, device='cuda:0')
tensor(420.4635, device='cuda:0')
tensor(328.1499, device='cuda:0')
tensor(353.3926, device='cuda:0')
tensor(247.4022, device='cuda:0')
tensor(270.9248, device='cuda:0')
tensor(374.0406, device='cuda:0')
tensor(372.1169, device='cuda:0')
tensor(240.3438, device='cuda:0')
tensor(257.2220, device='cuda:0')
tensor(262.2682, device='cuda:0')
tensor(184.5726, device='cuda:0')
tensor(269.8996, device='cuda:0')
tensor(296.4832, device='cuda:0')
tensor(226.1192, device='cuda:0')
tensor(223.3875, device='cuda:0')
tensor(245.1852, device='cuda:0')
tensor(164.1450, device='cuda:0')
tensor(247.5465, device='cuda:0')
tensor(222.7758, device='cuda:0')
tensor(512.6537, device='cuda:0')
tensor(359.1945, device='cuda:0')
tensor(195.0766, device='cuda:0')
tensor(292.0749, device='cuda:0')
tensor(218.6568, device='cuda:0')
tensor(326.718

In [ ]:
# Load your final model, that we will use for the rest of the PSET.
if early_stopping:
  weights_file = save_dir + '/weights_best_val_acc.pt'
else:
  weights_file = save_dir + '/weights_last.pt'
model = ImageModel()
model = model.to(device)
model.load_state_dict(torch.load(weights_file))

<All keys matched successfully>

In [ ]:
def evaluate(model, dataloader, device):

    '''
    A function to return the predicted age and true age (for evaluation). 
    
    Args: 
        model (torch.nn.Module): model 
        dataloader (torch.utils.data.Dataloader): DataLoader object for the test data
        device (str): Your device
        
    Returns: 
        (np.array, np.array): true age, predicted age and mean loss
    '''
    pred_list = []
    y_list = []
    mean_loss = []
    with torch.no_grad():
        model.eval()                
        for batch in tqdm(dataloader):
            epoch_loss = []
            image = batch['image'].to(device)
            age = batch['age'].to(device)
            pred = model(image).squeeze()
            mean_loss.append((pred-age).pow(2).mean().cpu())
            pred_list.extend(list(pred.cpu().numpy()))
            y_list.extend(list(age.cpu().numpy()))
            
            # evaluate your model here
    return y_list, pred_list, np.array(mean_loss).mean()

In [ ]:
y_list, pred_list, loss = evaluate(model, test_dataloader, device)
print(loss)

  0%|          | 0/176 [00:00<?, ?it/s]

324.86905


In [ ]:
# Dropout  = 0.25, test_mse = 324.82, num_epochs = 45
# Dropout = 0.3, test_mse = 364.57, num_epochs = 55

In [ ]:
def apply_dropout(m):
    if type(m) == nn.Dropout:
        m.train()

model.eval()

model.apply(apply_dropout)

In [ ]:
y_list_list = []
pred_list_list = []
loss_list = []
model.eval()
# generate uncertainty interval for dropout. Change to 200 if you have more time. Takes 5 hours to run
# on colab. Another approach is to run in several iteration and keep saving the result offline and later
# merging the predictions. Need to make sure, every time dataloader is run in same order, else can't 
# merge results. For this, we need to make sure that data from kaggle is divided into train and test and validation
# set in same way everytime we run the notebook. To achieve this, I saved the ids which went into each of the sets
# and split the data always in same way.
for i in tqdm(range(25)):
  y_list, pred_list, loss = evaluate(model, test_dataloader, device)
  y_list_list.append(y_list)
  pred_list_list.append(pred_list)
  loss_list.append(loss)
  print(loss_list[i])

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/176 [00:00<?, ?it/s]

323.1596


  0%|          | 0/176 [00:00<?, ?it/s]

319.846


...

320.44446


  0%|          | 0/176 [00:00<?, ?it/s]

319.74423


  0%|          | 0/176 [00:00<?, ?it/s]

321.54666


In [ ]:
data_age = []
id_list = []
for data in test_dataloader:
  data_age.extend(list(data['age'].cpu().numpy()))
  id_list.extend(list(data['id'].cpu().numpy()))

In [ ]:
import pickle
import numpy as np

In [ ]:
with open('dropout_25_25_2.pkl', 'wb') as f:
  pickle.dump({"pred":pred_list_list, "age": y_list_list, "id": id_list}, f)
files.download('dropout_25_25_2.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

In [ ]:
# final_dict["pred"] = np.concatenate((np.array(dict_new_1["pred"]), np.array(dict_new_2["pred"]),
#                 ), axis = 0)
# final_dict["age"] = dict_new_2["age"]
# final_dict["id"] = dict_new_2["id"]
# with open('dropout_25_200.pkl', 'wb') as f:
#   pickle.dump(final_dict, f)
# files.download('dropout_25_200.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Saved datafile of 200 runs on .3 dropout.
with open('dropout_30_200.pkl', 'rb') as f:
   final_dict = pickle.load(f)


In [ ]:
def build_quantile(array, quantile_value):
  """
  Builds quantile for an n_vals X n_sample size array for given quantile value (lower_quantile, upper_quantile)
  Returns 2 X n_sample quantiles
  """
  array = np.sort(array, axis = 0)
  length = array.shape[0]
  indices = (round(quantile_value[0]*length)-1, round(quantile_value[1]*length)-1)
  
  return array[indices, :]

def get_coverage(quantile_array, target_array):
  """
  Returns coverage for a quantile array and true value array input
  
  """
  return np.mean(np.logical_and((quantile_array[0, :]<=target_array),(quantile_array[1, :]>=target_array)))

def get_quantile_width(quantile_array):
  return np.mean(quantile_array[1]-quantile_array[0])

def get_qhat(quantile_array, target_array, alpha):
  n = len(target_array)
  arr = np.zeros(quantile_array.shape)
  arr[0, :] = np.array(quantile_array[0, :] - target_array)
  arr[1, :] = np.array(target_array - quantile_array[1, :])                  
  
  arr = arr.max(axis=0)
  return np.quantile(arr, np.ceil((n+1)*(1-alpha))/n)

def conformilize_quantiles(quantile_array, qhat):
  arr = np.zeros(quantile_array.shape)
  arr[0, :] = np.array(quantile_array[0, :] - qhat)
  arr[1, :] = np.array(quantile_array[1, :] + qhat)
  return arr

In [ ]:
def generate_conformal_sets(quantile_array, target, alpha,
                            caliberation_size = 500, n_simulation=10000):
  conformal_coverage = []
  quantile_width = []
  naive_coverage = []
  qhat_list = []
  for i in range(n_simulation):
    indices = np.arange(len(target))
    np.random.shuffle(indices)
    target = np.array(target)[indices]
    quantile_array = quantile_array[:, indices]
    caliberation_quantiles = quantile_array[:, 0:caliberation_size]
    test_quantiles = quantile_array[:, caliberation_size:]
    caliberation_target = target[0:caliberation_size]
    test_target = target[caliberation_size:]
    naive_coverage.append(get_coverage(test_quantiles, test_target))
    qhat = get_qhat(caliberation_quantiles, caliberation_target, alpha = alpha)
    qhat_list.append(qhat)
    conformal_quantiles = conformilize_quantiles(test_quantiles, qhat)
    conformal_coverage.append(get_coverage(conformal_quantiles, test_target))
    quantile_width.append(get_quantile_width(conformal_quantiles))
  mean_qhat = round(np.mean(np.abs(np.array(qhat_list))), 3)
  std_qhat = round(np.std(np.abs(np.array(qhat_list))), 3)
  print("Mean and std for qhat for alpha {} : {} and {} resp.".format(alpha, mean_qhat, std_qhat))
  return quantile_width, conformal_coverage, naive_coverage


In [ ]:
def plotting_func(quantile_width, naive_coverage, conformal_coverage, bins = 20):
  figure, axis = plt.subplots(1, 3, figsize=(16,4))
 
  axis[0].hist(quantile_width, bins = bins)
  axis[0].set_title("Conformal Quantile Width")
    
  axis[1].hist(naive_coverage, bins = bins)
  axis[1].set_title("Naive Dropout Coverage")
    
  axis[2].hist(conformal_coverage, bins = bins)
  axis[2].set_title("Conformalized Coverage")
  plt.show()
  print("The Quantile width is {} and std is {}".format
      (round(np.array(quantile_width).mean(), 3), round(np.array(quantile_width).std(), 3)))
  print("The Mean Naive coverage is {} with a std of {}".format(
      round(np.array(naive_coverage).mean(), 3), round(np.array(naive_coverage).std(), 3)))
  print("The Mean Conformal coverage is {} with a std of {}".format(
      round(np.array(conformal_coverage).mean(), 3), round(np.array(conformal_coverage).std(), 3)))
  
  print("")

In [ ]:
def build_df(final_dict, df, alpha):
  dropout_quantiles = build_quantile(final_dict["pred"], (alpha/2, 1-alpha/2))
  pred_df = pd.DataFrame(data=(dropout_quantiles.transpose()
                              ), columns=["quantiles_lower", "quantiles_upper"])
  pred_df["id"] = final_dict["id"]
  pred_df["age"] = final_dict["age"]
  pred_df = pred_df.merge(df[["id", "male"]], how = "inner", on = ["id"])
  pred_df = pred_df.set_index(["id"])
  return pred_df

In [ ]:
def get_conditional_coverage_gender(df, alpha, caliberation_size = 500, n_simulation=100):
  """
  df:Dataframe with columns "age", "male", "qantiles_lower", "quantiles_upper" 
  alpha: error rate
  caliberation_size = 500
  n_simulations
  returns: width of predict set, conformal coverage and naive coverage for (male, female)
  """
  conformal_coverage = []
  quantile_width = []
  naive_coverage = []
  qhat_list = []
  for i in range(n_simulation):
    df = df.sample(frac = 1)
    
    gender_array = np.array(df.male)
    target = np.array(list(df["age"]))

    quantile_array = np.array((df.quantiles_lower, df.quantiles_upper))
    caliberation_quantiles = quantile_array[:, 0:caliberation_size]
    test_quantiles = quantile_array[:, caliberation_size:]
    caliberation_target = target[0:caliberation_size]
    test_target = target[caliberation_size:]
    gender_array = gender_array[caliberation_size:]

    naive_coverage_male = get_coverage(test_quantiles[:, (gender_array==True)],
                                       test_target[(gender_array==True)])
    naive_coverage_female = get_coverage(test_quantiles[:, (gender_array==False)],
                                       test_target[(gender_array==False)])
    naive_coverage.append([naive_coverage_male, naive_coverage_female])
    qhat = get_qhat(caliberation_quantiles, caliberation_target, alpha = alpha)
    qhat_list.append(qhat)
    conformal_quantiles = conformilize_quantiles(test_quantiles, qhat)

    conformal_coverage_male = get_coverage(conformal_quantiles[:, (gender_array==True)],
                                       test_target[(gender_array==True)])
    conformal_coverage_female = get_coverage(conformal_quantiles[:, (gender_array==False)],
                                       test_target[(gender_array==False)])
    conformal_coverage.append([conformal_coverage_male, conformal_coverage_female])

    quantile_width_male = get_quantile_width(conformal_quantiles[:, (gender_array==True)])
                                       
    quantile_width_female = get_quantile_width(conformal_quantiles[:, (gender_array==False)])
    quantile_width.append([quantile_width_male, quantile_width_female])
  mean_qhat = round(np.mean(np.abs(np.array(qhat_list))), 3)
  print("Mean qhat for alpha {} : {} ".format(alpha, mean_qhat))
  return quantile_width, conformal_coverage, naive_coverage
